In [ ]:
import os
from datetime import datetime, timedelta
import ipywidgets as widgets
import plotly.graph_objs as go
import yfinance as yf
import pandas as pd
from IPython.display import display

In [ ]:
views = {
    "Raw Data": lambda x, y: x,
    "Percent Change": lambda x, y: x.pct_change(),
    "Rolling Average": lambda x, y: x.rolling(y).mean(),
    "Rolling Variance": lambda x, y: x.rolling(y).var(),
    "Rolling Standard Deviation": lambda x, y: x.rolling(y).var() ** 0.5,
    "Rolling Coefficient of Variation": lambda x, y: (x.rolling(y).var() ** 0.5)
    / (x.rolling(y).mean()),
}


def format_plotly(fig, calc, data, start, end):
    fig.update_yaxes(title=None)
    fig.update_xaxes(title=None)
    start_t = start.strftime("%Y/%m/%d")
    end_t = end.strftime("%Y/%m/%d")
    if calc:
        if len(calc) == 1:
            fig_title = f"{calc[0]} of {data} from {start_t} to {end_t}"
        else:
            fig_title = f"{', '.join(calc)} of {data} from {start_t} to {end_t}"
    fig.update_layout(
        autosize=False,
        width=1000,
        height=500,
        title={
            "text": fig_title,
            "y": 0.95,
            "x": 0.5,
            "xanchor": "center",
            "yanchor": "top",
        },
    )


last_tickers = ""
last_interval = "id"

In [ ]:
def view_chart(calculation, data, rolling, start, end, interval, tickers):
    global last_tickers, df, last_interval
    # set_interval_opts(interval_widget,start,end)
    if tickers and tickers[-1] == ",":
        if tickers != last_tickers or interval != last_interval:
            if interval in ["1d", "5d", "1wk", "1mo", "3mo"]:
                df = yf.download(
                    tickers, period="max", interval=interval, progress=False
                )
            else:
                df = yf.download(
                    tickers, start=start, end=end, interval=interval, progress=False
                )
            last_tickers = tickers

        start_n = datetime(start.year, start.month, start.day)
        end_n = datetime(end.year, end.month, end.day)
        fig = go.Figure()
        for item in calculation:
            calcs = views[item](df[data], rolling)
            if interval in ["1d", "5d", "1wk", "1mo", "3mo"]:
                result = calcs.loc[(calcs.index >= start_n) & (calcs.index <= end_n)]
            else:
                result = calcs

            if isinstance(result, pd.Series):
                name = f"{tickers.split(',')[0]} {item}"
                plot = go.Scatter(x=result.index, y=result, mode="lines", name=name)
                fig.add_trace(plot)
            else:
                for val in result:
                    name = f"{val.upper()} {item}"
                    plot = go.Scatter(
                        x=result.index,
                        y=result[val],
                        mode="lines",
                        name=name,
                        connectgaps=True,
                    )
                    fig.add_trace(plot)

        format_plotly(fig, calculation, data, start, end)

        """fig.update_layout(legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        ))"""

        if os.environ.get("SERVER_SOFTWARE", "jupyter").startswith("voila"):
            fig.show(config={"showTips": False}, renderer="notebook")
        else:
            fig.show(config={"showTips": False})


w_auto = widgets.Layout(width="auto")
calc_widget = widgets.SelectMultiple(
    options=list(views.keys()), value=["Raw Data"], layout=w_auto
)

data_opts = ["Open", "Close", "High", "Low", "Volume"]
data_widget = widgets.Dropdown(
    options=data_opts, value="Close", layout=w_auto, description="Data"
)
rolling_widget = widgets.Dropdown(
    options=list(range(2, 101)), value=60, layout=w_auto, description="Rolling"
)

base_date = (datetime.today() - timedelta(days=365)).date()
start_widget = widgets.DatePicker(value=base_date, layout=w_auto, description="Start")
end_widget = widgets.DatePicker(
    value=datetime.today().date(), layout=w_auto, description="End"
)
interval_opts = [
    "1m",
    "2m",
    "5m",
    "15m",
    "30m",
    "60m",
    "90m",
    "1h",
    "1d",
    "5d",
    "1wk",
    "1mo",
    "3mo",
]
interval_widget = widgets.Dropdown(
    options=interval_opts, value="1d", layout=w_auto, description="Interval"
)
tickers_widget = widgets.Textarea(
    value="TSLA,", layout=widgets.Layout(width="auto", height="100%")
)
data_box = widgets.VBox([data_widget, rolling_widget])
date_box = widgets.VBox([start_widget, end_widget, interval_widget])
controls = widgets.HBox(
    [tickers_widget, calc_widget, date_box, data_box],
    layout=widgets.Layout(width="90%"),
)

stocks_view = widgets.interactive_output(
    view_chart,
    {
        "calculation": calc_widget,
        "data": data_widget,
        "rolling": rolling_widget,
        "start": start_widget,
        "end": end_widget,
        "interval": interval_widget,
        "tickers": tickers_widget,
    },
)

title_html = "<h1>Stock Analysis Dashboard</h1>"
warning_html = (
    '<p style="color:red"=>You MUST use a comma after EVERY stock you type in.</p>'
)
app_contents = [
    widgets.HTML(title_html),
    controls,
    widgets.HTML(warning_html),
    stocks_view,
]
app = widgets.VBox(app_contents)
display(app)